In [1]:
using FermiCG
using PyCall
using Plots
using LinearAlgebra
using Printf

## Import PYSCF data

In [9]:
pyscf = pyimport("pyscf");
fcidump = pyimport("pyscf.tools.fcidump");
ctx = fcidump.read("fe2s2");
h = ctx["H1"];
g = ctx["H2"];
ecore = ctx["ECORE"];
g = pyscf.ao2mo.restore("1", g, size(h,2));

In [10]:
# blocking from paper
blocks = [[0,1],[2,3,4,5,6],[7,9,11],[8,10,12],[13,14,15,16,17],[18,19]]
init_fspace = ((2,2),(5,0),(3,3),(3,3),(5,0),(2,2))

for b in blocks
    b .+= 1
end
clusters = [Cluster(i,collect(blocks[i])) for i = 1:length(blocks)]
display(clusters)

((2, 2), (5, 0), (3, 3), (3, 3), (5, 0), (2, 2))

In [41]:
# blocking 1
blocks = [  [0,1,2,3,6,7,8],        # L1(sig)+ Fe1(t) + L2(sig)
            [11,12,13,16,17,18,19], # L2(sig)+ Fe2(t) + L3(sig)
            [4,5,9,10,14,15],       # Fe1(e) + Fe2(e) + L2(pi)
         ]

init_fspace = ((7,4), (7,4), (4,4))

for b in blocks
    b .+= 1
end
clusters = [Cluster(i,collect(blocks[i])) for i = 1:length(blocks)]
display(clusters)


3-element Array{Cluster,1}:
 Cluster(1, [1, 2, 3, 4, 7, 8, 9])
 Cluster(2, [12, 13, 14, 17, 18, 19, 20])
 Cluster(3, [5, 6, 10, 11, 15, 16])

In [68]:
ints = InCoreInts(ecore,h,g);

na = 0
nb = 0
for f in init_fspace
    na += f[1]
    nb += f[2]
end
display(na)
display(nb)
# get initial guess for 1RDM
tmp = eigen(ints.h1).vectors;
rdm1a_guess = tmp[:,1:na]*tmp[:,1:na]';
rdm1b_guess = tmp[:,1:nb]*tmp[:,1:nb]';
# eigen(ints.h1).values


18

12

In [87]:

e_cmf, U, Da, Db  = FermiCG.cmf_oo(ints, clusters, init_fspace, rdm1a_guess, rdm1b_guess,
                                        max_iter_oo=10, verbose=0, gconv=1e-6, method="bfgs");
ints2 = FermiCG.orbital_rotation(ints,U);


 ooCMF Iter:    1 Total= -115.877567420665 Active= -115.877567420665 G=     8.73e-01
 ooCMF Iter:    2 Total= -116.127452249661 Active= -116.127452249661 G=     5.87e-01
 ooCMF Iter:    3 Total= -116.296874821862 Active= -116.296874821862 G=     3.30e-01
 ooCMF Iter:    4 Total= -116.371647277115 Active= -116.371647277115 G=     1.50e-01
 ooCMF Iter:    5 Total= -116.373593160830 Active= -116.373593160830 G=     1.58e-01
 ooCMF Iter:    6 Total= -116.373476787237 Active= -116.373476787237 G=     1.78e-01


 * Status: failure (objective increased between iterations)

 * Candidate solution
    Final objective value:     -1.163734e+02

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 4.61e-02 ≰ 0.0e+00
    |x - x'|/|x'|          = 4.45e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.16e-04 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.00e-06 ≰ 0.0e+00
    |g(x)|                 = 3.95e-02 ≰ 1.0e-06

 * Work counters
    Seconds run:   42  (vs limit Inf)
    Iterations:    6
    f(x) calls:    123
    ∇f(x) calls:   123


*ooCMF -116.37336041 


┌ Warning:  energy increased
└ @ FermiCG /Users/nmayhall/code/FermiCG/src/CMFs.jl:724


In [88]:

e_cmf, U, Da, Db  = FermiCG.cmf_oo(ints2, clusters, init_fspace, Da, Db,
                                        max_iter_oo=40, verbose=0, gconv=1e-6, method="bfgs");


 ooCMF Iter:    1 Total= -116.373360401172 Active= -116.373360401172 G=     1.76e-01
 ooCMF Iter:    2 Total= -116.386971263498 Active= -116.386971263498 G=     6.73e-02
 ooCMF Iter:    3 Total= -116.390706391680 Active= -116.390706391680 G=     4.62e-02
 ooCMF Iter:    4 Total= -116.395416170957 Active= -116.395416170957 G=     6.64e-02
 ooCMF Iter:    5 Total= -116.402968079166 Active= -116.402968079166 G=     6.02e-02
 ooCMF Iter:    6 Total= -116.406108043379 Active= -116.406108043379 G=     3.64e-02
 ooCMF Iter:    7 Total= -116.407256662454 Active= -116.407256662454 G=     2.82e-02
 ooCMF Iter:    8 Total= -116.408192867290 Active= -116.408192867290 G=     2.34e-02
 ooCMF Iter:    9 Total= -116.408470285004 Active= -116.408470285004 G=     1.66e-02
 ooCMF Iter:   10 Total= -116.408683014419 Active= -116.408683014419 G=     1.81e-02
 ooCMF Iter:   11 Total= -116.408995670425 Active= -116.408995670425 G=     2.21e-02
 ooCMF Iter:   12 Total= -116.409473971890 Active= -116.409473971

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     -1.164200e+02

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 4.79e-03 ≰ 0.0e+00
    |x - x'|/|x'|          = 8.92e-03 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.58e-05 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.36e-07 ≰ 0.0e+00
    |g(x)|                 = 1.42e-03 ≰ 1.0e-06

 * Work counters
    Seconds run:   39  (vs limit Inf)
    Iterations:    40
    f(x) calls:    121
    ∇f(x) calls:   121


 ooCMF Iter:   41 Total= -116.419972233053 Active= -116.419972233053 G=     5.67e-03
*ooCMF -116.41997223 


In [96]:
ints3 = FermiCG.orbital_rotation(ints2,U);
e_cmf, U, Da, Db  = FermiCG.cmf_oo(ints3, clusters, init_fspace, Da, Db,
                                        max_iter_oo=40, verbose=0, gconv=1e-6, method="gd");


 ooCMF Iter:    1 Total= -116.419997218255 Active= -116.419997218255 G=     4.69e-03
 ooCMF Iter:    2 Total= -116.420008419930 Active= -116.420008419930 G=     2.65e-03
 ooCMF Iter:    3 Total= -116.420015395097 Active= -116.420015395097 G=     1.95e-03
 ooCMF Iter:    4 Total= -116.420019034652 Active= -116.420019034652 G=     2.17e-03
 ooCMF Iter:    5 Total= -116.420022708966 Active= -116.420022708966 G=     1.81e-03
 ooCMF Iter:    6 Total= -116.420024968278 Active= -116.420024968278 G=     2.09e-03
 ooCMF Iter:    7 Total= -116.420028482955 Active= -116.420028482955 G=     1.76e-03
 ooCMF Iter:    8 Total= -116.420030605708 Active= -116.420030605708 G=     2.06e-03
 ooCMF Iter:    9 Total= -116.420034087088 Active= -116.420034087088 G=     1.74e-03
 ooCMF Iter:   10 Total= -116.420036132172 Active= -116.420036132172 G=     2.04e-03
 ooCMF Iter:   11 Total= -116.420039611459 Active= -116.420039611459 G=     1.73e-03
 ooCMF Iter:   12 Total= -116.420041597616 Active= -116.420041597

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     -1.164201e+02

 * Found with
    Algorithm:     Gradient Descent

 * Convergence measures
    |x - x'|               = 8.21e-04 ≰ 0.0e+00
    |x - x'|/|x'|          = 2.93e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 3.29e-06 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.83e-08 ≰ 0.0e+00
    |g(x)|                 = 4.54e-04 ≰ 1.0e-06

 * Work counters
    Seconds run:   34  (vs limit Inf)
    Iterations:    40
    f(x) calls:    101
    ∇f(x) calls:   101


 ooCMF Iter:   41 Total= -116.420121031157 Active= -116.420121031157 G=     1.74e-03
*ooCMF -116.42012103 


In [97]:
ints4 = FermiCG.orbital_rotation(ints3,U);
e_cmf, U, Da, Db  = FermiCG.cmf_oo(ints4, clusters, init_fspace, Da, Db,
                                        max_iter_oo=40, verbose=0, gconv=1e-6, method="bfgs");

 ooCMF Iter:    1 Total= -116.420121033129 Active= -116.420121033129 G=     1.74e-03
 ooCMF Iter:    2 Total= -116.420123260869 Active= -116.420123260869 G=     2.03e-03
 ooCMF Iter:    3 Total= -116.421180328172 Active= -116.421180328172 G=     2.37e-02
 ooCMF Iter:    4 Total= -116.421260239944 Active= -116.421260239944 G=     1.15e-02
 ooCMF Iter:    5 Total= -116.421307963231 Active= -116.421307963231 G=     3.27e-03
 ooCMF Iter:    6 Total= -116.421330574064 Active= -116.421330574064 G=     3.65e-03
 ooCMF Iter:    7 Total= -116.421387711744 Active= -116.421387711744 G=     6.08e-03
 ooCMF Iter:    8 Total= -116.421432539951 Active= -116.421432539951 G=     5.06e-03
 ooCMF Iter:    9 Total= -116.421446692755 Active= -116.421446692755 G=     3.42e-03
 ooCMF Iter:   10 Total= -116.421462146655 Active= -116.421462146655 G=     3.05e-03
 ooCMF Iter:   11 Total= -116.421466891017 Active= -116.421466891017 G=     2.98e-03
 ooCMF Iter:   12 Total= -116.421469990404 Active= -116.421469990

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     -1.164216e+02

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 2.19e-03 ≰ 0.0e+00
    |x - x'|/|x'|          = 4.35e-03 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.80e-06 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.55e-08 ≰ 0.0e+00
    |g(x)|                 = 3.98e-04 ≰ 1.0e-06

 * Work counters
    Seconds run:   40  (vs limit Inf)
    Iterations:    40
    f(x) calls:    123
    ∇f(x) calls:   123


 ooCMF Iter:   41 Total= -116.421555268925 Active= -116.421555268925 G=     1.24e-03
*ooCMF -116.42155527 


In [98]:
ints5 = FermiCG.orbital_rotation(ints4,U);
e_cmf, U, Da, Db  = FermiCG.cmf_oo(ints5, clusters, init_fspace, Da, Db,
                                        max_iter_oo=40, verbose=0, gconv=1e-6, method="bfgs");

 ooCMF Iter:    1 Total= -116.421555273756 Active= -116.421555273756 G=     1.24e-03
 ooCMF Iter:    2 Total= -116.421556318396 Active= -116.421556318396 G=     6.81e-04
 ooCMF Iter:    3 Total= -116.421556630364 Active= -116.421556630364 G=     6.09e-04
 ooCMF Iter:    4 Total= -116.421558695974 Active= -116.421558695974 G=     6.22e-04
 ooCMF Iter:    5 Total= -116.421558568476 Active= -116.421558568476 G=     5.66e-04
 ooCMF Iter:    6 Total= -116.421559369245 Active= -116.421559369245 G=     7.26e-04
 ooCMF Iter:    7 Total= -116.421561672578 Active= -116.421561672578 G=     7.33e-04
 ooCMF Iter:    8 Total= -116.421561844918 Active= -116.421561844918 G=     7.89e-04
 ooCMF Iter:    9 Total= -116.421563764474 Active= -116.421563764474 G=     7.60e-04
 ooCMF Iter:   10 Total= -116.421564802688 Active= -116.421564802688 G=     7.14e-04
 ooCMF Iter:   11 Total= -116.421564658260 Active= -116.421564658260 G=     6.97e-04
 ooCMF Iter:   12 Total= -116.421565100894 Active= -116.421565100

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     -1.164217e+02

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 4.56e-03 ≰ 0.0e+00
    |x - x'|/|x'|          = 2.09e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.28e-06 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.96e-08 ≰ 0.0e+00
    |g(x)|                 = 3.31e-04 ≰ 1.0e-06

 * Work counters
    Seconds run:   40  (vs limit Inf)
    Iterations:    40
    f(x) calls:    122
    ∇f(x) calls:   122


 ooCMF Iter:   41 Total= -116.421722799613 Active= -116.421722799613 G=     1.33e-03
*ooCMF -116.42172280 


In [129]:
ints5 = FermiCG.orbital_rotation(ints5,U);
e_cmf, U, Da, Db  = FermiCG.cmf_oo(ints5, clusters, init_fspace, Da, Db,
                                        max_iter_oo=20, verbose=0, gconv=1e-6, method="bfgs");

 ooCMF Iter:    1 Total= -116.421763398626 Active= -116.421763398626 G=     4.18e-06
 ooCMF Iter:    2 Total= -116.421763397020 Active= -116.421763397020 G=     4.80e-06
 ooCMF Iter:    3 Total= -116.421763374366 Active= -116.421763374366 G=     6.82e-06
 ooCMF Iter:    4 Total= -116.421763371962 Active= -116.421763371962 G=     3.93e-06
 ooCMF Iter:    5 Total= -116.421763344951 Active= -116.421763344951 G=     6.52e-06
 ooCMF Iter:    6 Total= -116.421763354302 Active= -116.421763354302 G=     6.17e-06
 ooCMF Iter:    7 Total= -116.421763472835 Active= -116.421763472835 G=     1.83e-05
 ooCMF Iter:    8 Total= -116.421763465982 Active= -116.421763465982 G=     2.98e-05
 ooCMF Iter:    9 Total= -116.421763369930 Active= -116.421763369930 G=     2.32e-05
 ooCMF Iter:   10 Total= -116.421763420328 Active= -116.421763420328 G=     2.57e-05
 ooCMF Iter:   11 Total= -116.421763537481 Active= -116.421763537481 G=     3.56e-05
 ooCMF Iter:   12 Total= -116.421763473078 Active= -116.421763473

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     -1.164218e+02

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 4.46e-04 ≰ 0.0e+00
    |x - x'|/|x'|          = 4.66e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 5.20e-08 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 4.46e-10 ≰ 0.0e+00
    |g(x)|                 = 6.71e-06 ≰ 1.0e-06

 * Work counters
    Seconds run:   21  (vs limit Inf)
    Iterations:    20
    f(x) calls:    68
    ∇f(x) calls:   68


 ooCMF Iter:   21 Total= -116.421763507888 Active= -116.421763507888 G=     2.91e-05
*ooCMF -116.42176351 
